In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['SimHei']   # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False   # 用来正常显示负号

# 加载提供的Excel文件
file_path_1 = '.\附件1.xlsx'
file_path_2 = '.\附件2.xlsx'

# 从两个文件中读取工作表
land_data = pd.read_excel(file_path_1, sheet_name='乡村的现有耕地')
crop_data = pd.read_excel(file_path_1, sheet_name='乡村种植的农作物')
planting_data_2023 = pd.read_excel(file_path_2, sheet_name='2023年的农作物种植情况')
stats_data_2023 = pd.read_excel(file_path_2, sheet_name='2023年统计的相关数据')

# 2023年农作物种植分布
plt.figure(figsize=(12, 8))
crop_distribution = planting_data_2023['作物类型'].value_counts()
colors = sns.color_palette('Set2', len(crop_distribution))
sns.barplot(x=crop_distribution.index, y=crop_distribution.values, hue=crop_distribution.index, palette=colors, legend=False)

def pastel_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return sns.color_palette('pastel')[random_state.randint(0, len(sns.color_palette('pastel')) - 1)]
plt.title('2023年农作物种植分布', fontsize=16)
plt.xlabel('作物类型', fontsize=14)
plt.ylabel('数量', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()


plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file_path_1 = '.\附件1.xlsx'
file_path_2 = '.\附件2.xlsx'

# 从两个文件中读取工作表
land_data = pd.read_excel(file_path_1, sheet_name='乡村的现有耕地')
crop_data = pd.read_excel(file_path_1, sheet_name='乡村种植的农作物')
planting_data_2023 = pd.read_excel(file_path_2, sheet_name='2023年的农作物种植情况')
stats_data_2023 = pd.read_excel(file_path_2, sheet_name='2023年统计的相关数据')

# 确保销售单价列是字符串类型
stats_data_2023['销售单价/(元/斤)'] = stats_data_2023['销售单价/(元/斤)'].astype(str)

# 然后计算平均销售单价，同时处理可能的格式错误
stats_data_2023['平均销售单价/元'] = stats_data_2023['销售单价/(元/斤)'].apply(
    lambda x: (float(x.split('-')[0]) + float(x.split('-')[1])) / 2 if '-' in x and len(x.split('-')) == 2 else None
)
# 2. 作物产量分析
plt.figure(figsize=(16, 8))
plt.subplot(1, 3, 1)
sns.barplot(x='作物名称', y='亩产量/斤', data=stats_data_2023, hue='作物名称', legend=False)
plt.xlabel('作物名称', fontsize=14)
plt.ylabel('产量 (斤/亩)', fontsize=14)
plt.xticks(rotation=45)

# 3. 种植成本分析
plt.subplot(1, 3, 2)
sns.barplot(x='作物名称', y='种植成本/(元/亩)', data=stats_data_2023, hue='作物名称', legend=False)
plt.xlabel('作物名称', fontsize=14)
plt.ylabel('种植成本 (元/亩)', fontsize=14)
plt.xticks(rotation=45)

# 4. 销售价格分析
plt.subplot(1, 3, 3)
sns.barplot(x='作物名称', y='平均销售单价/元', data=stats_data_2023, hue='作物名称', legend=False)
plt.xlabel('作物名称', fontsize=14)
plt.ylabel('平均销售单价 (元/斤)', fontsize=14)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
from scipy.optimize import linprog


file_path_1 = '.\附件1.xlsx'
file_path_2 = '.\附件2.xlsx'

# 从两个文件中读取工作表
land_data = pd.read_excel(file_path_1, sheet_name='乡村的现有耕地')
crop_data = pd.read_excel(file_path_1, sheet_name='乡村种植的农作物')
planting_data_2023 = pd.read_excel(file_path_2, sheet_name='2023年的农作物种植情况')
stats_data_2023 = pd.read_excel(file_path_2, sheet_name='2023年统计的相关数据')

# 提取参数
P = stats_data_2023['亩产量/斤'].values
C = stats_data_2023['种植成本/(元/亩)'].values
S = stats_data_2023['销售单价/(元/斤)'].str.split('-', expand=True).astype(float).mean(axis=1).values
Q = planting_data_2023.groupby('作物编号')['种植面积/亩'].sum().values * P

N = len(P)  # 作物种类数量
M = land_data['地块面积/亩'].sum()  # 总耕地面积


print(P)
print(C)
print(S)
print(Q)
print(N)
print(M)


# # 定义目标函数系数（以最大化收益为目标，需要转换为最小化损失问题）
# 	# 相关代码

# # 定义约束条件
# A = [P]  # 耕地面积限制
# b = [M]  # 最大耕地面积

# # 边界条件
# x_bounds = [(0, None)] * N

# # 求解情况1：滞销
# 	# 相关代码

# # 求解情况2：降价销售
# 	# 相关代码

# print('最优种植方案（情况1）：', result1.x)
# print('最优种植方案（情况2）：', result2.x)



In [ ]:
# 可视化代码
import matplotlib.pyplot as plt

# 情况1和情况2的最优种植方案比较
plt.figure(figsize=(12, 6))
	# 相关代码

plt.xlabel('作物编号')
plt.ylabel('种植面积 (亩)')
plt.title('不同情况的最优种植方案')
plt.legend()
plt.show()

In [ ]:
from deap import base, creator, tools, algorithms
import pandas as pd
import numpy as np
import multiprocessing

# Global variables to store data
fields = None
crops = None
stats_2023 = None
years = range(2024, 2031)
seasons = ["spring", "autumn"]

def load_data():
    global fields, crops, stats_2023
    # Your data loading code
    attachment1_path = "./附件1.xlsx"
    attachment2_path = "./附件2.xlsx"

    fields = pd.read_excel(attachment1_path, sheet_name=0)
    fields = fields.rename(
        columns={"地块名称": "name", "地块类型": "type", "地块面积/亩": "area"}
    )

    crops = pd.read_excel(attachment1_path, sheet_name=1)
    crops = crops.rename(
        columns={
            "作物编号": "id",
            "作物名称": "name",
            "作物类型": "type",
            "种植耕地": "suitable_fields",
        }
    )

    planting_2023 = pd.read_excel(attachment2_path, sheet_name=0)
    planting_2023 = planting_2023.rename(
        columns={
            "种植地块": "field",
            "作物编号": "crop_id",
            "作物名称": "crop_name",
            "作物类型": "crop_type",
            "种植面积/亩": "area",
            "种植季次": "season",
        }
    )

    stats_2023 = pd.read_excel(attachment2_path, sheet_name=1)
    stats_2023 = stats_2023.rename(
        columns={
            "序号": "index",
            "作物编号": "crop_id",
            "作物名称": "crop_name",
            "地块类型": "field_type",
            "种植季次": "season",
            "亩产量/斤": "yield_per_mu",
            "种植成本/(元/亩)": "cost_per_mu",
            "销售单价/(元/斤)": "price_range",
        }
    )
    stats_2023[["min_price", "max_price"]] = (
        stats_2023["price_range"].str.split("-", expand=True).astype(float)
    )
    stats_2023["avg_price"] = (stats_2023["min_price"] + stats_2023["max_price"]) / 2
    stats_2023 = stats_2023.drop("price_range", axis=1)

class Context:
    def __init__(self, fields, crops, stats_2023):
        self.fields = fields
        self.crops = crops
        self.stats_2023 = stats_2023

def evaluate(individual):
    global fields, crops, stats_2023
    num_crops = len(crops)
    num_fields = len(fields)
    num_years = len(years)
    num_seasons = len(seasons)

    plant_matrix = np.array(individual).reshape(
        (num_crops, num_fields, num_years, num_seasons)
    )

    profit = 0
    crop_stats_dict = stats_2023.set_index('crop_id').to_dict('index')
    field_area_dict = fields.set_index('name')['area'].to_dict()

    for c in range(num_crops):
        crop_id = crops.iloc[c]["id"]
        if crop_id not in crop_stats_dict:
            continue

        crop_stats = crop_stats_dict[crop_id]
        yield_per_mu = crop_stats["yield_per_mu"]
        avg_price = crop_stats["avg_price"]
        cost_per_mu = crop_stats["cost_per_mu"]

        for f in range(num_fields):
            field_name = fields.iloc[f]["name"]
            field_area = field_area_dict.get(field_name, 0)

            for y in range(num_years):
                for s in range(num_seasons):
                    area = plant_matrix[c, f, y, s]
                    if area > field_area:
                        area = field_area  # Ensure we do not exceed field area
                    revenue = area * yield_per_mu * avg_price
                    cost = area * cost_per_mu
                    profit += revenue - cost

    return (profit,)

def main():
    load_data()
    num_crops = len(crops)
    num_fields = len(fields)
    num_years = len(years)
    num_seasons = len(seasons)
    num_genes = num_crops * num_fields * num_years * num_seasons

    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register("attr_float", np.random.uniform, 0, 100)
    toolbox.register(
        "individual",
        tools.initRepeat,
        creator.Individual,
        toolbox.attr_float,
        n=num_genes,
    )
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    context = Context(fields, crops, stats_2023)
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register(
        "mutate", tools.mutUniformInt, low=0, up=100, indpb=0.2
    )
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(n=50)
    ngen = 10
    cxpb = 0.5
    mutpb = 0.2

    # Set up multiprocessing pool
    pool = multiprocessing.Pool()
    toolbox.register("map", pool.map)
    algorithms.eaSimple(
        population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, verbose=True
    )
    pool.close()
    pool.join()

    best_individual = tools.selBest(population, 1)[0]
    print(f"Best individual is: {best_individual}")

if __name__ == "__main__":
    main()


In [ ]:
import pulp as pl
import pandas as pd
import numpy as np
df11 = pd.read_excel("./附件1.xlsx", sheet_name="乡村的现有耕地")
df22 = pd.read_excel("./附件1.xlsx", sheet_name="乡村种植的农作物")
#地块名称
dkmc=df11['地块名称'].tolist()
# 种植地方、种植作物类型、年份
zw = df22['作物名称'].tolist()
years=['2024','2025','2026','2027','2028','2029','2030']
seaborn=[1,2]
# 豆类
doulei=[1,2,3,4,5,17,18,19]
#读入数据
df1 = pd.read_excel("./附件2_1.xlsx", sheet_name="2023年的农作物种植情况")
df2 = pd.read_excel("./附件2_1.xlsx", sheet_name="2023年统计的相关数据")
#农作物亩产量
acr=df2['亩产量/斤']
#农作物单价
danjia=df2['销售单价/(元/斤)']
#农作物种植成本
chengben=df2['种植成本/(元/亩)']
#种植面积
zzmj=df2['种植面积/亩']
mbxl=acr*zzmj
#第一小问
model_one =pl.LpProblem('pro_one',pl.LpMaximize)
#决策变量
x = pl.LpVariable.dicts('Area',(dkmc,zw,years),lowBound=0,cat='Integer')
#目标函数：最大化利润
if(acr*x[dkmc][zw][years][seaborn]<=mbxl):
    max_profit1=pl.lpSum([
        danjia*pl.lpSum(acr*x[dkmc][zw][years][seaborn])])
else:
    max_profit1 = pl.lpSum([
        danjia * pl.lpSum(acr * mbxl)])
model_one+=max_profit1
#约束
for i in dkmc:
    for j in years:
        for k in seaborn:
            model_one+=pl.lpSum([x[i][crop][j][k]for crop in dkmc])<=dkmc[i]
for i in dkmc:
    for j in years:
        model_one+=pl.pSum([x[i][j][season] for i in doulei for season in seaborn])>=zzmj[i]/3
for i in dkmc:
    for j in years[:-1]:
        for k in seaborn:
            for l in acr:
                model_one += x[i][j][k][l]
x[i][j+1][k][l]==0
model_one.solve()
 
# 第二种情况
model_two = pl.LpProblem("Max_Profit2", lp.LpMaximize)
 
x=pl.LpVariable.dicts("Area",("plots","crops","years","seasons"), lowBound=0,cat='Integer')
 
if(acr*x[dkmc][zw][years][seaborn]<=mbxl):
    max_profit1=pl.lpSum([
        danjia*pl.lpSum(acr*x[dkmc][zw][years][seaborn])])
else:
    max_profit1 = pl.lpSum([
        danjia * pl.lpSum(acr * mbxl)])+0.5(danjia-mbxl)*pl.lpSum([ danjia * pl.lpSum(acr * mbxl)])
model_two+=max_profit1
 
#约束
for i in dkmc:
    for j in years:
        for k in seaborn:
            model_one+=pl.lpSum([x[i][crop][j][k]for crop in dkmc])<=dkmc[i]
for i in dkmc:
    for j in years:
        model_one+=pl.pSum([x[i][j][season] for i in doulei for season in seaborn])>=zzmj[i]/3
for i in dkmc:
    for j in years[:-1]:
        for k in seaborn:
            for l in acr:
                model_one += x[i][j][k][l]
x[i][j+1][k][l]==0
model_two.solve()
print('输出结果',model_two.solve())
print('输出结果',model_two.solve())

In [ ]:
from deap import base, creator, tools, algorithms
import pandas as pd
import numpy as np

def load_data():
    # 设置文件路径
    attachment1_path = "./附件1.xlsx"
    attachment2_path = "./附件2.xlsx"

    # 读取第一个表格：乡村的现有耕地
    fields = pd.read_excel(attachment1_path, sheet_name=0)
    fields = fields.rename(
        columns={"地块名称": "name", "地块类型": "type", "地块面积/亩": "area"}
    )

    # 读取第二个表格：乡村种植的农作物
    crops = pd.read_excel(attachment1_path, sheet_name=1)
    crops = crops.rename(
        columns={
            "作物编号": "id",
            "作物名称": "name",
            "作物类型": "type",
            "种植耕地": "suitable_fields",
        }
    )

    # 读取第三个表格：2023年的农作物种植情况
    planting_2023 = pd.read_excel(attachment2_path, sheet_name=0)
    planting_2023 = planting_2023.rename(
        columns={
            "种植地块": "field",
            "作物编号": "crop_id",
            "作物名称": "crop_name",
            "作物类型": "crop_type",
            "种植面积/亩": "area",
            "种植季次": "season",
        }
    )

    # 读取第四个表格：2023年统计的相关数据
    stats_2023 = pd.read_excel(attachment2_path, sheet_name=1)
    stats_2023 = stats_2023.rename(
        columns={
            "序号": "index",
            "作物编号": "crop_id",
            "作物名称": "crop_name",
            "地块类型": "field_type",
            "种植季次": "season",
            "亩产量/斤": "yield_per_mu",
            "种植成本/(元/亩)": "cost_per_mu",
            "销售单价/(元/斤)": "price_range",
        }
    )
    # 处理价格范围
    stats_2023[["min_price", "max_price"]] = (
        stats_2023["price_range"].str.split("-", expand=True).astype(float)
    )
    stats_2023["avg_price"] = (stats_2023["min_price"] + stats_2023["max_price"]) / 2
    stats_2023 = stats_2023.drop("price_range", axis=1)

    return fields, crops, planting_2023, stats_2023

def create_evaluate_function(fields, crops, stats_2023, years, seasons):
    def evaluate(individual):
        num_crops = len(crops)
        num_fields = len(fields)
        num_years = len(years)
        num_seasons = len(seasons)

        # Reshape individual into a 4D matrix (crops, fields, years, seasons)
        plant_matrix = np.array(individual).reshape(
            (num_crops, num_fields, num_years, num_seasons)
        )

        profit = 0
        for c in range(num_crops):
            for f in range(num_fields):
                for y in range(num_years):
                    for s in range(num_seasons):
                        area = plant_matrix[c, f, y, s]
                        crop_id = crops.iloc[c]["id"]
                        field_name = fields.iloc[f]["name"]
                        # Get crop and field data
                        crop_stats = stats_2023[stats_2023["crop_id"] == crop_id]
                        if crop_stats.empty:
                            continue
                        yield_per_mu = crop_stats["yield_per_mu"].values[0]
                        avg_price = crop_stats["avg_price"].values[0]
                        cost_per_mu = crop_stats["cost_per_mu"].values[0]
                        field_area = fields[fields["name"] == field_name][
                            "area"
                        ].values[0]
                        # Calculate profit
                        if area > field_area:
                            area = field_area  # Ensure we do not exceed field area
                        revenue = area * yield_per_mu * avg_price
                        cost = area * cost_per_mu
                        profit += revenue - cost
        return (profit,)
    
    return evaluate

def main():
    # Load data
    fields, crops, planting_2023, stats_2023 = load_data()
    years = range(2024, 2031)
    seasons = ["spring", "autumn"]
    num_crops = len(crops)
    num_fields = len(fields)
    num_years = len(years)
    num_seasons = len(seasons)
    num_genes = num_crops * num_fields * num_years * num_seasons

    # Define the problem
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Maximize profit
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox()
    toolbox.register(
        "attr_float", np.random.uniform, 0, 100
    )  # Random float between 0 and 100
    toolbox.register(
        "individual",
        tools.initRepeat,
        creator.Individual,
        toolbox.attr_float,
        n=num_genes,
    )
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Create the evaluate function with the necessary data
    evaluate = create_evaluate_function(fields, crops, stats_2023, years, seasons)
    
    toolbox.register("evaluate", evaluate)
    toolbox.register("mate", tools.cxBlend, alpha=0.5)
    toolbox.register(
    "mutate", tools.mutUniformInt, low=0, up=100, indpb=0.2
)
    toolbox.register("select", tools.selTournament, tournsize=3)

    population = toolbox.population(
        n=50
    )  # Create an initial population of 50 individuals
    ngen = 10  # Number of generations
    cxpb = 0.5  # Crossover probability
    mutpb = 0.2  # Mutation probability

    algorithms.eaSimple(
        population, toolbox, cxpb=cxpb, mutpb=mutpb, ngen=ngen, verbose=True
    )

    # Extract the best individual
    best_individual = tools.selBest(population, 1)[0] 
    print(f"Best individual is: {best_individual}")

if __name__ == "__main__":
    main()
